In [1]:
from sqlalchemy import create_engine
from datetime import date

USER = 'capstone'
PASSWORD = 'capstone'
HOST = 'localhost'
DATABASE = 'debt_management'

mariadb_connection_uri = f'mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{DATABASE}'

engine = create_engine(mariadb_connection_uri, future=True)

from sqlalchemy import text

from tkinter import *
from tkinter import ttk
import tkinter
root=Tk()
root.title('Debt Ledger')
frm = ttk.Frame(root, padding=20)
frm.grid()

#LIST for sql account names

with engine.connect() as conn: 

    name_list = []

    get_name_row = "SELECT * FROM accounts;"
    get_name_row = conn.execute(text(get_name_row))

    for row in get_name_row.fetchall(): #https://stackoverflow.com/questions/14835852/convert-sql-result-to-list-python
        name_list.append(row[1])  
    
#select person with existing debt
introlabel0=ttk.Label(frm, text="Check debt of person with existing debt")
introlabel0.grid(row=0,column=0,sticky=W)

def selectperson():
    #INPUTS 
    name = selectedperson_var.get()
    
    with engine.connect() as conn:
        
        #GET the row that the id number corresponds to
        get_row = "SELECT * FROM accounts WHERE name LIKE '%"+name+"%';"
        get_row = conn.execute(text(get_row))
        row = get_row.fetchone() #https://stackoverflow.com/questions/13205157/just-one-row-from-sqlalchemy
        
        #GET quantity from the values in row
        debt = str(row[2]) 
        
        #GET id_no
        get_row = "SELECT * FROM accounts WHERE name LIKE '%"+name+"%';"
        get_row = conn.execute(text(get_row))
        row = get_row.fetchone()
        
        id_no = str(row[0])
        
        result_str = name+ " [id:" +str(id_no)+ "] currently has " +debt+ " in debt"
    
    result = ttk.Label(frm, text=result_str)
    result.grid(row=3,column=0,sticky=W,pady=2)      

selectedperson_var=StringVar()
selectedperson_var.set('Select Name')#defult empty
#selectedperson_var.grid(column=0,row=2,sticky=W,pady=2)

drop1=OptionMenu(frm,selectedperson_var,*name_list)
drop1.grid(column=0, row=1,sticky=W,pady=2)

receivepaymentppl_var=StringVar()

drop2=OptionMenu(frm,receivepaymentppl_var,*name_list)

adddebtname_var=StringVar()

drop3=OptionMenu(frm,adddebtname_var,*name_list)

submitselectperson=Button(frm,text='Submit',command=selectperson)
submitselectperson.grid(column=1,row=2,sticky=E,pady=2)

currentdebt=('current debt')

div0=ttk.Label(frm,text='----------------------------------------------------------------------------------------------------------')
div0.grid(column=0,row=4)
#add person with debt

def addpeople():
    #INPUTS
    name = addedperson_entry.get()
    debt_amount = debtfromnewppl_entry.get()
    
    with engine.connect() as conn:
        
        #INSERT new account into table  #https://www.w3schools.com/sql/sql_insert.asp
        new = "INSERT INTO accounts (name, amount) VALUES ('" +name+ "'," +str(debt_amount)+ ");"
        new = conn.execute(text(new))
        
        conn.commit()
    
        #GET id_no
        get_row = "SELECT * FROM accounts WHERE name LIKE '%"+name+"%';"
        get_row = conn.execute(text(get_row))
        row = get_row.fetchone()
        
        id_no = str(row[0])
        
        #LIST for sql account names and refresh all options

        name_list = []
        drop1.children["menu"].delete(0,"end")
        drop2.children["menu"].delete(0,"end")
        drop3.children["menu"].delete(0,"end")

        get_name_row = "SELECT * FROM accounts;"
        get_name_row = conn.execute(text(get_name_row))

        for row in get_name_row.fetchall(): #https://stackoverflow.com/questions/14835852/convert-sql-result-to-list-python
            name_list.append(row[1])  
            
            drop1.children["menu"].add_command(label=row[1], command=lambda x = row[1]: selectedperson_var.set(x))
            drop2.children["menu"].add_command(label=row[1], command=lambda x =row[1]: receivepaymentppl_var.set(x)) 
            drop3.children["menu"].add_command(label=row[1], command=lambda x =row[1]: adddebtname_var.set(x))
        
        selectedperson_var.set('Select Name')
        receivepaymentppl_var.set('Select Name')
        adddebtname_var.set('Select Name')
        
        conn.close()
        
        result_str = "A new account has been made for " +name+ " [id:" +str(id_no)+ "] who currently has " +str(debt_amount)+ " in debt"
    
    result = ttk.Label(frm, text=result_str)
    result.grid(row=9,column=0,sticky=W,pady=2)

introlabel1=ttk.Label(frm, text="Adding New People")
introlabel1.grid(row=5,column=0,sticky=W,pady=2)
addingpeople_label = ttk.Label(frm, text="Name")
addingpeople_label.grid(row=6,column=0,sticky=W,pady=2)
addingpeople_var = StringVar()
addedperson_entry = ttk.Entry(frm, textvariable=addingpeople_var)
addedperson_entry.grid(column=1, row=6,sticky=E,pady=2)

debtfromnewppl_label = ttk.Label(frm, text="Amount")
debtfromnewppl_label.grid(row=7,column=0,sticky=W,pady=2)
debtfromnewppl_var = DoubleVar()
debtfromnewppl_entry= ttk.Entry(frm, textvariable=debtfromnewppl_var)
debtfromnewppl_entry.grid(column=1, row=7,sticky=E,pady=2)

addpplsubmit = ttk.Button(frm, text="Submit", command=addpeople)
addpplsubmit.grid(column=1, row=8)

div1=ttk.Label(frm,text='----------------------------------------------------------------------------------------------------------')

div1.grid(column=0,row=11)
#receiving payment

introlabel2=ttk.Label(frm, text="Receiving Payment")
introlabel2.grid(row=12,column=0,sticky=W,pady=2)

def receivepayment():
    
    #INPUTS
    amount_paid = paymentreceived_entry.get()
    name = receivepaymentppl_var.get()
receivepaymentppl_var.set('Select Name')
    paymentreceived_entry.set('')
        
    with engine.connect() as conn:
        #Change values #https://www.w3schools.com/sql/sql_update.asp
        
        pay = "UPDATE accounts SET amount = amount -" +str(amount_paid)+ " WHERE name LIKE '%"+name+"%';"
        pay = conn.execute(text(pay))
        
        conn.commit()
        
        #GET current_balance
        get_row = "SELECT * FROM accounts WHERE name LIKE '%"+name+"%';"
        get_row = conn.execute(text(get_row))
        row = get_row.fetchone()
        current_balance = str(row[2])
        
        #GET id_no
        get_row = "SELECT * FROM accounts WHERE name LIKE '%"+name+"%';"
        get_row = conn.execute(text(get_row))
        row = get_row.fetchone()
        
        id_no = str(row[0])
        
        result_str = str(amount_paid)+ " was received from " +name+ " [id:" +str(id_no)+ "].\nCurrent Debt Balance: " + current_balance

    result=ttk.Label(frm,text=result_str)
    result.grid(row=16,column=0,sticky=W,pady=2)   

receivepaymentppl_var.set('Select Name')
drop2.grid(column=0, row=13,sticky=W,pady=2)

receivedpayment_label = ttk.Label(frm, text="Amount")
receivedpayment_label.grid(row=14,column=0,sticky=W,pady=2)
paymentreceived_var = DoubleVar()
paymentreceived_entry= ttk.Entry(frm, textvariable=paymentreceived_var)
paymentreceived_entry.grid(column=1, row=14,sticky=E,pady=2)

submitreceivepayment=ttk.Button(frm,text='Submit',command=receivepayment)
submitreceivepayment.grid(column=1,row=15,sticky=E,pady=2)
div1=ttk.Label(frm,text='----------------------------------------------------------------------------------------------------------')

div1.grid(column=0,row=18)
#incurring more debt
introlabel3=ttk.Label(frm, text="Incurring More Debt")
introlabel3.grid(row=19,column=0,sticky=W,pady=2)
def adddebtincurred():
    
    #INPUTS
    name = adddebtname_var.get()
    debt_increase = addboramnt_entry.get()
    addboramnt_var.set('')
     adddebtname_var.set('Select Name')
    
    with engine.connect() as conn:  
        
        #Change values #https://www.w3schools.com/sql/sql_update.asp
        inc = "UPDATE accounts SET amount = amount +" +str(debt_increase)+ " WHERE name LIKE '%"+name+"%';"
        inc = conn.execute(text(inc))
        
        conn.commit()
        
        #GET current_balance
        get_row = "SELECT * FROM accounts WHERE name LIKE '%"+name+"%';"
        get_row = conn.execute(text(get_row))
        row = get_row.fetchone()
        current_balance = str(row[2])
        
        #GET id_no
        get_row = "SELECT * FROM accounts WHERE name LIKE '%"+name+"%';"
        get_row = conn.execute(text(get_row))
        row = get_row.fetchone()
        
        id_no = str(row[0])
        
        result_str = str(debt_increase)+ " was added to the account of " +name+ " [id:" +str(id_no)+ "].\nCurrent Debt Balance: " + current_balance

    result=ttk.Label(frm,text=result_str)
    result.grid(row=23,column=0,sticky=W,pady=2)   
        
adddebtname_var.set('Select Name')
drop3.grid(column=0, row=20,sticky=W,pady=2)


addboramnt_label = ttk.Label(frm, text="Amount")
addboramnt_label.grid(row=21,column=0,sticky=W,pady=2)
addboramnt_var = DoubleVar()
addboramnt_entry= ttk.Entry(frm, textvariable=addboramnt_var)
addboramnt_entry.grid(column=1, row=20,sticky=E,pady=2)

submitadddebt=ttk.Button(frm,text='Submit',command=adddebtincurred)
submitadddebt.grid(column=1,row=22,sticky=E,pady=2)

'''
TABLES #https://stackhowto.com/how-to-open-a-new-window-with-a-button-in-python-tkinter/
'''

div1=ttk.Label(frm,text='----------------------------------------------------------------------------')
div1.grid(column=0,row=25)
introlabel4=ttk.Label(frm, text="Table of People with Debt")
introlabel4.grid(row=26,column=0,sticky=W,pady=2)

def table_people_with_debt(): #https://pythonguides.com/python-tkinter-table-tutorial/
    table = Tk()
    table.title('Table of People with Debt')
    
    table_frame = Frame(table)
    table_frame.pack()
    
    #scrollbar
    scroll = Scrollbar(table_frame)
    scroll.pack(side=RIGHT, fill=Y)
    
    table1 = ttk.Treeview(table_frame, yscrollcommand = scroll.set, xscrollcommand = scroll.set)
    
    table1.pack()

    scroll.config(command=table1.yview)
    
    table1['columns'] = ('ID', 'NAME', 'DEBT AMOUNT')
    
    table1.column("#0", width=0,  stretch=NO)
    table1.column("ID",anchor=CENTER, width=80)
    table1.column("NAME",anchor=CENTER,width=80)
    table1.column("DEBT AMOUNT",anchor=CENTER,width=80)
    
    table1.heading("#0",text="",anchor=CENTER)
    table1.heading("ID",text="Id",anchor=CENTER)
    table1.heading("NAME",text="Name",anchor=CENTER)
    table1.heading("DEBT AMOUNT",text="DEBT AMOUNT",anchor=CENTER)
    
    
    with engine.connect() as conn:
        
        #QUERY - all debts that are > than 0
        current = "SELECT * FROM accounts WHERE amount > 0"
        current = conn.execute(text(current))
        row_data = current.fetchall()
        
        #QUERY - number of rows
        row_no_sql = "SELECT COUNT(*) FROM accounts WHERE amount > 0"
        row_no_sql = conn.execute(text(row_no_sql))
        row_no = row_no_sql.fetchone()
        
        i = 0
        for id, name, amount in row_data:
            table1.insert(parent='',index='end',iid=i,text='',values=(id,name,amount))
            i += 1
    
    table1.pack()

    
submitadddebt=ttk.Button(frm,text='Access Table Result',command=table_people_with_debt)
submitadddebt.grid(column=0,row=28,sticky=W,pady=2)
    

div1=ttk.Label(frm,text='----------------------------------------------------------------------------')
div1.grid(column=0,row=32)
introlabel4=ttk.Label(frm, text="Table of Payment Received")
introlabel4.grid(row=33,column=0,sticky=W,pady=2)



div1=ttk.Label(frm,text='----------------------------------------------------------------------------')
div1.grid(column=0,row=39)
introlabel4=ttk.Label(frm, text="Table of Additional Debt Incurred")
introlabel4.grid(row=40,column=0,sticky=W,pady=2)

root.mainloop()